In [12]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [13]:
# Create network
model = Network('model')

# Block 1: nodes
node = np.empty(3, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.PS)
node[2] = Queue(model, 'Queue2', SchedStrategy.PS)

# Block 2: classes (1 job each)
jobclass = np.empty(2, dtype=object)
jobclass[0] = ClosedClass(model, 'Class1', 1, node[0], 0)
jobclass[1] = ClosedClass(model, 'Class2', 1, node[0], 0)

print("Network created with 3 nodes and 2 classes (1 job each)")
print(f"  {node[0].get_name()}: Delay node")
print(f"  {node[1].get_name()}: PS Queue")
print(f"  {node[2].get_name()}: PS Queue")

Network created with 3 nodes and 2 classes (1 job each)
  Delay: Delay node
  Queue1: PS Queue
  Queue2: PS Queue


In [14]:
# Set advanced service distributions# Delay node - mixed distributionsnode[0].set_service(jobclass[0], Erlang.fitMeanAndOrder(3, 2))  # Erlang with mean=3, shape=2node[0].set_service(jobclass[1], HyperExp(0.5, 3.0, 10.0))     # HyperExp with prob=0.5, rates=3.0,10.0print("Delay node distributions:")print("  Class1: Erlang(mean=3, shape=2) - Low variability")print("  Class2: HyperExp(p=0.5, λ1=3, λ2=10) - High variability")# Queue1 - HyperExp and MMPP2node[1].set_service(jobclass[0], HyperExp(0.1, 1.0, 10.0))     # HyperExp with prob=0.1, rates=1.0,10.0node[1].set_service(jobclass[1], MMPP2(1, 2, 3, 4))           # MMPP2 with parameters λ0=1, λ1=2, σ01=3, σ10=4print("\nQueue1 distributions:")print("  Class1: HyperExp(p=0.1, λ1=1, λ2=10) - Very high variability")print("  Class2: MMPP2(λ0=1, λ1=2, σ01=3, σ10=4) - Markov Modulated Poisson Process")# Queue2 - HyperExp and Erlangnode[2].set_service(jobclass[0], HyperExp(0.1, 1.0, 10.0))     # Same as Queue1 for Class1node[2].set_service(jobclass[1], Erlang(1, 2))                 # Erlang with rate=1, shape=2print("\nQueue2 distributions:")print("  Class1: HyperExp(p=0.1, λ1=1, λ2=10) - Same as Queue1")print("  Class2: Erlang(rate=1, shape=2) - Low variability")

=== Setting Advanced Service Distributions ===
Delay node distributions:
  Class1: Erlang(mean=3, shape=2) - Low variability
  Class2: HyperExp(p=0.5, λ1=3, λ2=10) - High variability

Queue1 distributions:
  Class1: HyperExp(p=0.1, λ1=1, λ2=10) - Very high variability
  Class2: MMPP2(λ0=1, λ1=2, σ01=3, σ10=4) - Markov Modulated Poisson Process

Queue2 distributions:
  Class1: HyperExp(p=0.1, λ1=1, λ2=10) - Same as Queue1
  Class2: Erlang(rate=1, shape=2) - Low variability


In [15]:
# Set up connectivity
model.add_link(node[0], node[0])  # Delay can route to itself
model.add_link(node[0], node[1])  # Delay -> Queue1
model.add_link(node[0], node[2])  # Delay -> Queue2
model.add_link(node[1], node[0])  # Queue1 -> Delay
model.add_link(node[2], node[0])  # Queue2 -> Delay

print("\n=== Configuring Routing Strategies ===")

# Class1: Probabilistic routing from Delay
node[0].setProbRouting(jobclass[0], node[0], 0.0)  # No self-loop for Class1
node[0].setProbRouting(jobclass[0], node[1], 0.3)  # 30% to Queue1
node[0].setProbRouting(jobclass[0], node[2], 0.7)  # 70% to Queue2
node[1].setProbRouting(jobclass[0], node[0], 1.0)  # Queue1 -> Delay (deterministic)
node[2].setProbRouting(jobclass[0], node[0], 1.0)  # Queue2 -> Delay (deterministic)

print("Class1 routing (Probabilistic):")
print("  From Delay: 30% Queue1, 70% Queue2")
print("  From Queues: 100% back to Delay")

# Class2: Random routing strategy
node[0].set_routing(jobclass[1], RoutingStrategy.RAND)  # Random from Delay
node[1].set_routing(jobclass[1], RoutingStrategy.RAND)  # Random from Queue1
node[2].set_routing(jobclass[1], RoutingStrategy.RAND)  # Random from Queue2

print("\nClass2 routing (Random):")
print("  All nodes use random routing among connected destinations")


=== Configuring Routing Strategies ===
Class1 routing (Probabilistic):
  From Delay: 30% Queue1, 70% Queue2
  From Queues: 100% back to Delay

Class2 routing (Random):
  All nodes use random routing among connected destinations


In [16]:
# Link the routing matrix
P = model.init_routing_matrix()
model.link(P)

print("Routing matrix linked successfully")
print("\nModel configuration complete!")

Routing matrix linked successfully

Model configuration complete!


Jul 30, 2025 11:15:43 PM jline.io.InputOutputKt line_printf



In [ ]:
# Aligned with JAR test scenarios for cqn_mmpp2_service# JAR tests: JMT(seed=23000) only# Configure solver (using JMT as it handles advanced distributions well)solver = SolverJMT(model, seed=23000)print("Solver configured with seed=23000 (matches JAR)")

In [18]:
# Solve the model
print(f'\n=== SOLVER: {solver.get_name()} ===')
avg_table = solver.get_avg_table()
print(avg_table)


=== SOLVER: SolverJMT ===
JMT Model: /tmp/workspace/jsim/17557913342183253771/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 1.008300s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   Class1  0.9101  0.9101  2.9593  2.9593  0.3071  0.3072
1   Delay   Class2  0.1870  0.1870  0.2174  0.2174  0.8582  0.8583
2  Queue1   Class1  0.0208  0.0170  0.2280  0.0684  0.0908  0.0908
3  Queue1   Class2  0.2022  0.1992  0.7076  0.2359  0.2880  0.2876
4  Queue2   Class1  0.0685  0.0421  0.3070  0.2149  0.2155  0.2155
5  Queue2   Class2  0.6105  0.5942  2.0809  0.6936  0.2929  0.2953
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   Class1  0.9101  0.9101  2.9593  2.9593  0.3071  0.3072
1   Delay   Class2  0.1870  0.1870  0.2174  0.2174  0.8582  0.8583
2  Queue1   Class1  0.0208  0.0170  0.2280  0.0684  0.0908  0.0908
3  Queue1   Class2  0.2022  0.1992  0.7076  0.2359  0.2880  0.2876
4  Queue2   Class1  0.0685  0.0421